In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import json
from datetime import datetime, time

MIN_SHADOW_PROPORTION = 0.0001  # 0.0005

In [ ]:
# Assume that the UNIX time key range for the data is 0930 to 1600 UTC, and does not need to fit into a US time zone, either EST or EDT.

# Load JSON data
with open('/content/drive/MyDrive/Colab Notebooks/futures.ai/spy_1min_regularhours.json', 'r') as json_file:
    json_data = json.load(json_file)

# Convert to DataFrame
df = pd.DataFrame(json_data)

# Calculate EMA
def calculate_ema(df, period):
    return df['close'].ewm(span=period, adjust=False).mean()

# Revised RSI calculation to uses EMA
def calculate_rsi(df, period=7):
    delta = df['close'].diff()

    # Separate gains and losses
    gains = delta.where(delta > 0, 0.0)
    losses = -delta.where(delta < 0, 0.0)

    # Calculate EMA multiplier
    alpha = 2 / (period + 1)

    # First averages (SMA)
    first_avg_gain = gains.iloc[:period].mean() if period <= len(gains) else gains.mean()
    first_avg_loss = losses.iloc[:period].mean() if period <= len(losses) else losses.mean()

    # Initialize series
    avg_gains = pd.Series(index=delta.index)
    avg_losses = pd.Series(index=delta.index)

    # Set first values
    avg_gains.iloc[period-1] = first_avg_gain
    avg_losses.iloc[period-1] = first_avg_loss

    # Calculate EMA
    for i in range(period, len(gains)):
        avg_gains.iloc[i] = alpha * gains.iloc[i] + (1 - alpha) * avg_gains.iloc[i-1]
        avg_losses.iloc[i] = alpha * losses.iloc[i] + (1 - alpha) * avg_losses.iloc[i-1]

    rs = avg_gains / np.where(avg_losses == 0, 1e-10, avg_losses)
    rsi = 100 - (100 / (1 + rs))

    # Ensure RSI stays within bounds
    rsi = np.clip(rsi, 0, 100)

    return avg_gains, avg_losses, rsi

# Append EMA to DataFrame
df['EMA_5'] = calculate_ema(df, 5)
df['EMA_10'] = calculate_ema(df, 10)
df['EMA_15'] = calculate_ema(df, 15)
df['EMA_20'] = calculate_ema(df, 20)

# Calculate RSI and gain/loss
df['Gain'], df['Loss'], df['RSI'] = calculate_rsi(df)

# Drop the first 20 rows (EMA 20)
df = df.iloc[20:].reset_index(drop=True)

# Check for NaN values in the DataFrame
if df.isna().any().any():
    print("Error: NaN values found in calculations.")
    problematic_rows = df[df.isna().any(axis=1)]
    print(problematic_rows[['time_key', 'close', 'RSI']])  # Print only relevant columns

    # Additional diagnostics
    print("Close prices for problematic rows:")
    for index in problematic_rows.index:
        print(f"Index: {index}, Close Price: {df.at[index, 'close']}, Gain: {df.at[index, 'Gain']}, Loss: {df.at[index, 'Loss']}")

# Count rows before dropping
initial_row_count = df.shape[0]

# Drop rows where any of the indicators are NaN
df = df.dropna(subset=['EMA_5', 'EMA_10', 'EMA_15', 'EMA_20', 'RSI']).reset_index(drop=True)

# Count rows after dropping
final_row_count = df.shape[0]
dropped_rows_count = initial_row_count - final_row_count

# Print the number of dropped rows
print(f"Number of rows dropped due to NaN values: {dropped_rows_count}")

# Convert time_key to datetime
df['datetime'] = pd.to_datetime(df['time_key'], unit='s')

# Filter out days that do not start before 9:35 AM
df['date'] = df['datetime'].dt.date
# Convert '09:35:00' to a datetime object for comparison with datetime
comparison_time = pd.to_datetime('09:35:00').time()
# Check the minimum time for each date and compare
valid_days = df.groupby('date')['datetime'].min().dt.time < comparison_time
valid_dates = valid_days[valid_days].index
# Filter the DataFrame to keep only valid dates
df = df[df['date'].isin(valid_dates)].copy()

# Filter function for trading hours
def is_valid_trading_time(dt):
    # Keep data between 10:00 AM and 3:55 PM
    return time(10, 0) <= dt.time() <= time(15, 55) # For 3 minute interval: time(10, 30) <= t <= time(15, 55)

# Apply time filter directly on the datetime column
df = df[df['datetime'].apply(is_valid_trading_time)].copy()

# Transform RSI to Int values based on specified rules
def transform_rsi_to_int(rsi):
    if rsi >= 70:
        return 1
    elif rsi <= 30:
        return -1
    else:
        return 0

df['RSI_INT'] = df['RSI'].apply(transform_rsi_to_int)

# Add EMA comparison columns
df['EMA_5_EMA_10'] = (df['EMA_5'] > df['EMA_10']).astype(int)
df['EMA_15_EMA_20'] = (df['EMA_15'] > df['EMA_20']).astype(int)

# Calculate shadow lengths
def calculate_shadow_fold(row):
    # body_size = abs(row['open'] - row['close'])
    min_body_threshold = row['close'] * MIN_SHADOW_PROPORTION

    if row['open'] > row['close']:  # Bearish candle
        upper_shadow = row['high'] - row['open']
        lower_shadow = row['close'] - row['low']
    else:  # Bullish candle
        upper_shadow = row['high'] - row['close']
        lower_shadow = row['open'] - row['low']

    if upper_shadow < min_body_threshold:
        upper_fold = 0
    else:
        upper_fold = upper_shadow / min_body_threshold

    if lower_shadow < min_body_threshold:
        lower_fold = 0
    else:
        lower_fold = lower_shadow / min_body_threshold

    return upper_fold, lower_fold

# Calculate shadow folds for each row
shadow_folds = df.apply(calculate_shadow_fold, axis=1)
upper_folds, lower_folds = zip(*shadow_folds)

# Convert folds to categorical values (0, 1, 2, etc.)
def categorize_fold(fold):
    if fold < 1:
        return 0
    return int(fold)

df['LONG_UPPER_SHADOW'] = [categorize_fold(fold) for fold in upper_folds]
df['LONG_LOWER_SHADOW'] = [categorize_fold(fold) for fold in lower_folds]

# Drop the temporary columns
# df = df.drop(['open', 'high', 'low'], axis=1)

# Convert 'datetime' to ISO format (string)
df['datetime'] = df['datetime'].dt.strftime('%Y-%m-%dT%H:%M:%S')

# Retain fields along with calculated indicators
result = df[['datetime', 'close', 'LONG_UPPER_SHADOW', 'LONG_LOWER_SHADOW', 'EMA_5', 'EMA_10', 'EMA_15', 'EMA_20', 'EMA_5_EMA_10', 'EMA_15_EMA_20', 'RSI', 'RSI_INT']].copy() #'open', 'high', 'low',

# Convert to dictionary format for JSON
result = result.to_dict(orient='records')

# Save to a new JSON file
# with open('/content/drive/MyDrive/Colab Notebooks/futures.ai/spy_1min_regularhours_truncated_preprocessed.json', 'w') as json_file:
with open('/content/drive/MyDrive/Colab Notebooks/futures.ai/spy_1min_regularhours_truncated_semisupervised_preprocessed.json', 'w') as json_file:
    json.dump(result, json_file)

Number of rows dropped due to NaN values: 0
